# Chapter 5. Object References, Mutability, and Recycling
---

## ToC

[Objectives](#objectives)  

1. [Variables Are Not Boxes](#variables-are-not-boxes)
2. [Identity, Equality, and Aliases](#identity-equality-and-aliases)  
    2.1. [Choosing Between == and is](#choosing-between--and-is)  
    2.2. [The Relative Immutability of Tuples](#the-relative-immutability-of-tuples)

---

## Objectives

A name is not the object; a name is a separate thing. We start the chapter by presenting a metaphor for variables in Python: variables are labels, not boxes. We then discuss the concepts of object identity, value, and aliasing. A surprising trait of tuples is revealed: they are immutable but their values may change. This leads to a discussion of shallow and deep copies. References and function parameters are our
next theme: the problem with mutable parameter defaults and the safe handling of
mutable arguments passed by clients of our functions. The last sections of the chapter cover garbage collection, the del command, and a selection of tricks that Python plays with immutable objects.

## Variables Are Not Boxes

Python variables are like reference variables in Java; a better metaphor is to think of variables as labels with names attached to objects.

**Example:** In following, variables a and b hold references to the same list, not copies of the list

In [6]:
a = [1, 2, 3]
b = a
print("id(a):", id(a))
print("id(b):", id(b))
a.append(4)
b

id(a): 1945982105984
id(b): 1945982105984


[1, 2, 3, 4]

![Figure 85](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/85.PNG)

Therefore, the `b = a` statement does not copy the contents of box `a` into box `b`. It attaches the label `b` to the object that already has the label `a`. With reference variables, it makes much more sense to say that the variable is assigned to an object, and not the other way around. After all, the object is created before the assignment.

Since the verb “to assign” is used in contradictory ways, a useful alternative is “to
bind”: Python’s assignment statement `x = …` **binds** the x name to the object created
or referenced on the righthand side.

**Example 6-2** Variables are bound to objects only after the objects are created

In [2]:
class Gizmo:
    def __init__(self):
        print(f'Gizmo id: {id(self)}')

In [27]:
# the output Gizmo id: … is a side effect of creating a Gizmo instance.
x = Gizmo()

Gizmo id: 1945977298464


In [28]:
id(x)

1945977298464

Here is proof that a second Gizmo was actually instantiated before the multiplication
was attempted.

In [29]:
y = Gizmo() * 10 # type:ignore

Gizmo id: 1945975719552


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [18]:
dir()

['Gizmo',
 'In',
 'Out',
 '_',
 '_1',
 '_14',
 '_17',
 '_4',
 '_5',
 '_6',
 '_9',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__vsc_ipynb_file__',
 '_dh',
 '_i',
 '_i1',
 '_i10',
 '_i11',
 '_i12',
 '_i13',
 '_i14',
 '_i15',
 '_i16',
 '_i17',
 '_i18',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_i7',
 '_i8',
 '_i9',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'a',
 'b',
 'exit',
 'get_ipython',
 'open',
 'quit',
 'x']

But variable `y` was never created, because the exception happened while the
righthand side of the assignment was being evaluated. Another way to evidence this:


In [3]:
try:
    x = Gizmo()
    y = x * 10
except TypeError:
    print("Caught TypeError")

print('x' in locals())  # True
print('y' in locals())  # False

Gizmo id: 2418083323280
Caught TypeError
True
False


![Figure 86](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/86.PNG)

## Identity, Equality, and Aliases

In [1]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles

In [2]:
lewis is charles

True

In [3]:
id(charles), id(lewis)

(2330441178432, 2330441178432)

In [4]:
lewis['balance'] = 950

In [5]:
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

In [6]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

In [7]:
alex == charles

True

In [8]:
alex is not charles

True

In [9]:
print(id(alex))
print(id(charles))

2330441185280
2330441178432


![Figure 87](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/87.PNG)

Example above was an example of *aliasing*. `lewis` and `charles` are aliases:
two variables bound to the same object. On the other hand, `alex` is not an alias for `charles`. The objects bound to `alex` and `charles` have the same value—that's what `==` compares—but they have different
identities.

An object’s identity never changes once it has been created; you may think of it as the
object’s address in memory. The is operator compares the identity of two objects; the
`id()` function returns an integer representing its identity.

In practice, we rarely use the `id()` function while programming. Identity checks are
most often done with the `is` operator, which compares the object IDs, so our code
doesn’t need to call `id()` explicitly.

![Figure 88](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/88.PNG)

### Choosing Between == and is

An example of using `is` to create and test a sentinel
object:

```python
END_OF_DATA = object()
# ... many lines
def traverse(...):
    # ... more lines
    if node is END_OF_DATA:
        return
    # etc.
```

The is operator is faster than `==`, because it cannot be overloaded, so Python does
not have to find and invoke special methods to evaluate it, and computing is as simple
as comparing two integer IDs, so it produces the same result as `is`. But most built-in types override `__eq__` with more meaningful implementations that actually take into account the values of the object
attributes. Equality may involve a lot of processing—for example, when comparing
large collections or deeply nested structures.

In [11]:
class Foo:
    pass

x = Foo()
y = Foo()

print(x == y)  # False
print(x is y)  # False


False
False


In [12]:
a = [1, 2, 3]
b = [1, 2, 3]

print(a == b)  # True — compares contents
print(a is b)  # False — different objects


True
False


![Figure 89](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/89.PNG)

### The Relative Immutability of Tuples

Tuples, like most Python collections—lists, dicts, sets, etc.—are containers: they hold
references to objects. In contrast, flat sequences like `str`, `bytes`, and `array.array` don’t contain references but directly hold their
contents—characters, bytes, and numbers—in contiguous memory.

If the referenced items are mutable, they may change even if
the tuple itself does not. In other words, the immutability of tuples really refers to the
physical contents of the `tuple` data structure (i.e., the references it holds), and does
not extend to the referenced objects.

**Example**

In [24]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
t3 = t1
t1 == t2

True

In [25]:
id(t1[-1])

2330441098688

In [26]:
id(t2[-1])

2330441323584

In [27]:
id(t3[-1])

2330441098688

In [28]:
t1[-1].append(99)

In [29]:
t1

(1, 2, [30, 40, 99])

In [30]:
t3

(1, 2, [30, 40, 99])

The identity of `t1[-1]` has not changed, only its value.

In [31]:
id(t1[-1])

2330441098688

In [33]:
t1 == t2

False

This relative immutability of tuples is behind the riddle An example similar to `A += Assignment Puzzler` we saw in `Part_I/Chapter_02_ArrayOfSequences/03_Slicing_PlusAsteriksSigns.ipynb`. It’s also the reason why some tuples are unhashable, as we’ve seen in
“What Is Hashable” on `Part_I/Chapter_03_DictsSets/02_PatternMatchingMappings_APIofMappingTypes.ipynb`
